# Programming Assignment: Frequent Itemset Mining Using Apriori

## Pattern Discovery in Data Mining Course - Coursera

* Author: Michael Onishi
* Date: October 25th, 2019

##Description
In this programming assignment, you are required to implement the Apriori algorithm and apply it to mine frequent itemsets from a real-life data set.

###Input
The provided input file ("categories.txt") consists of the category lists of 77,185 places in the US. Each line corresponds to the category list of one place, where the list consists of a number of category instances (e.g., hotels, restaurants, etc.) that are separated by semicolons.

An example line is provided below:

Local Services;IT Services & Computer Repair

In the example above, the corresponding place has two category instances: "Local Services" and "IT Services & Computer Repair".

###Output
You need to implement the Apriori algorithm and use it to mine category sets that are frequent in the input data. When implementing the Apriori algorithm, you may use any programming language you like. We only need your result pattern file, not your source code file.

After implementing the Apriori algorithm, please set the relative minimum support to 0.01 and run it on the 77,185 category lists. In other words, you need to extract all the category sets that have an absolute support larger than 771.

In [1]:
! wget -O categories.txt "https://d3c33hcgiwev3.cloudfront.net/_9b0d0ff87935997de01d221fd74bae90_categories.txt?Expires=1572134400&Signature=F6r7BpI2rngHAiFXa58MCTGUczt-punzssytlUC6qjJQjgkdBnWwNAd0a0LDiLs2nPOV6YlPXESX1fuc7x4zsTVJONGubTQPl~aZ~A-xED6S7-4KxwZunqYTR6WoShC4lLm4fKYok3PgpIrtRV9~o6ev4~RPDvAA5oM91miMsmk_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

--2019-10-25 23:22:44--  https://d3c33hcgiwev3.cloudfront.net/_9b0d0ff87935997de01d221fd74bae90_categories.txt?Expires=1572134400&Signature=F6r7BpI2rngHAiFXa58MCTGUczt-punzssytlUC6qjJQjgkdBnWwNAd0a0LDiLs2nPOV6YlPXESX1fuc7x4zsTVJONGubTQPl~aZ~A-xED6S7-4KxwZunqYTR6WoShC4lLm4fKYok3PgpIrtRV9~o6ev4~RPDvAA5oM91miMsmk_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
Resolving d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)... 13.224.12.124, 13.224.12.5, 13.224.12.139, ...
Connecting to d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)|13.224.12.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2859008 (2.7M) [text/plain]
Saving to: ‘categories.txt’

categories.txt      100%[===================>]   2.73M  6.19MB/s    in 0.4s    

2019-10-25 23:22:45 (6.19 MB/s) - ‘categories.txt’ saved [2859008/2859008]



##Part 1

Please output all the length-1 frequent categories with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category and should be in the following format:

support:category

For example, suppose a category (Fast Food) has an absolute support 3000, then the line corresponding to this frequent category set in "patterns.txt" should be:

3000:Fast Food

In [0]:
import pandas as pd

In [0]:
records = []
with open('categories.txt') as in_file:
  for line in in_file:
    records.append(line.strip().split(";"))

In [4]:
minsup = 0.01 * len(records)
print(minsup)

771.85


In [0]:
from collections import defaultdict
d = defaultdict(int)
for r in records:
  for c in r:
    d[frozenset([c])] += 1

In [0]:
with open('patterns-part1.txt', 'w') as out_file:
  for k,v in list(d.items()):
    if v > minsup:
      out_file.write(f"{v}:{list(k)[0]}\n")
    else:
      d.pop(k)

##Part 2

Please write all the frequent category sets along with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category set and should be in the following format:

support:category_1;category_2;category_3;...

For example, suppose a category set (Fast Food; Restaurants) has an absolute support 2851, then the line corresponding to this frequent category set in "patterns.txt" should be:

2851:Fast Food;Restaurants

In [0]:
ans = [d]

In [0]:
def handle_invalid_chars(s):
  return s.replace(' ', '_').replace('&','AND').replace('(', 'YY').replace(')', 'ZZ').replace('\'', 'XX')

def revert_original(s):
  return s.replace('_', ' ').replace('AND','&').replace('YY', '(').replace('ZZ', ')').replace('XX', '\'')

In [0]:
row_list = []
for r in records:
  row_list.append({handle_invalid_chars(c) : 1 for c in r if frozenset([c]) in d})

In [0]:
df = pd.DataFrame(row_list, columns=[handle_invalid_chars(list(x)[0]) for x in d.keys()])

In [11]:
df.head()

,Breakfast_AND_Brunch,American_YYTraditionalZZ,Restaurants,Sandwiches,Local_Services,Italian,Food,Coffee_AND_Tea,Fast_Food,Home_Services,Real_Estate,Bars,Sports_Bars,Nightlife,American_YYNewZZ,Automotive,Grocery,Event_Planning_AND_Services,Shopping,Home_AND_Garden,Auto_Repair,Burgers,Pizza,Beauty_AND_Spas,Active_Life,Ice_Cream_AND_Frozen_Yogurt,Hair_Salons,Doctors,Health_AND_Medical,Arts_AND_Entertainment,Cafes,Bakeries,Hotels_AND_Travel,Hotels,Fitness_AND_Instruction,Nail_Salons,Fashion,Chinese,WomenXXs_Clothing,Pet_Services,Pets,Specialty_Food,Mexican,Pubs,Sushi_Bars,Japanese,Professional_Services,Financial_Services,Dentists,General_Dentistry
0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
len(ans[0].keys())

50

In [0]:
k = 1
while True:
  dk = defaultdict(int)
  keys = list(ans[k-1].keys())
  num_items = len(keys)
  for i in range(num_items):
    item1 = keys[i]
    for j in range(i+1, num_items):
      item2 = keys[j]
      diff = item2 - item1
      if len(diff) == 1:
        candidate = item1.union(diff)
        query = ' & '.join([f'{handle_invalid_chars(c)}==1' for c in candidate])
        if df.query(query).shape[0] >= minsup:
          #print(candidate)
          #print(df.query(query).shape[0])
          dk[candidate] = df.query(query).shape[0]
  if len(dk) > 0:
    ans.append(dk)
    k += 1
  else:
    break;

In [14]:
ans

[defaultdict(int,
             {frozenset({'Breakfast & Brunch'}): 1369,
              frozenset({'American (Traditional)'}): 2416,
              frozenset({'Restaurants'}): 25071,
              frozenset({'Sandwiches'}): 2364,
              frozenset({'Local Services'}): 3468,
              frozenset({'Italian'}): 1848,
              frozenset({'Food'}): 9250,
              frozenset({'Coffee & Tea'}): 2199,
              frozenset({'Fast Food'}): 2851,
              frozenset({'Home Services'}): 4785,
              frozenset({'Real Estate'}): 1424,
              frozenset({'Bars'}): 4328,
              frozenset({'Sports Bars'}): 818,
              frozenset({'Nightlife'}): 5088,
              frozenset({'American (New)'}): 1593,
              frozenset({'Automotive'}): 4208,
              frozenset({'Grocery'}): 1424,
              frozenset({'Event Planning & Services'}): 2975,
              frozenset({'Shopping'}): 11233,
              frozenset({'Home & Garden'}): 1586,
         

In [0]:
with open('patterns-part2.txt', 'w') as out_file:
  for items in ans:
    for k,v in list(items.items()):
      out_file.write(f"{v}:{';'.join(list(k))}\n")